[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/grounded-segment-anything-colab/blob/main/grounded-segment-anything-colab.ipynb)

In [ ]:
%cd /content
!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything
%cd /content/Grounded-Segment-Anything
!pip install -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install .
%cd /content/Grounded-Segment-Anything
!python gradio_app.py --share --debug